In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 ouput channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [6]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's weight

10
torch.Size([6, 1, 5, 5])


In [8]:
input = torch.randn(1, 1, 32, 32)
print(input)
out = net(input)
print(out)

tensor([[[[-0.0422, -2.0327,  0.9664,  ..., -0.2630,  0.0556,  0.2651],
          [-1.4731, -1.5046,  0.2619,  ...,  0.5974, -1.4926, -0.1837],
          [ 0.6746,  0.9824,  0.2678,  ..., -0.2053,  0.7043, -0.6575],
          ...,
          [-1.3010,  1.1899,  0.9582,  ..., -2.0009, -0.7406,  0.2111],
          [ 0.2798, -0.2020,  0.4990,  ..., -0.3063,  0.0157,  0.2469],
          [-1.2540,  0.2808, -0.0129,  ..., -1.5105,  0.6924,  1.0900]]]])
tensor([[ 0.0035, -0.0129,  0.0026,  0.0302, -0.2330, -0.0373,  0.0834,  0.0972,
         -0.0097, -0.1114]], grad_fn=<AddmmBackward0>)


In [9]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [14]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6638, grad_fn=<MseLossBackward0>)


In [16]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

In [18]:
net.zero_grad()
print('conv1.bias.gra before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.gra after backward')
print(net.conv1.bias.grad)

conv1.bias.gra before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.gra after backward
tensor([-0.0015,  0.0041, -0.0049,  0.0228,  0.0025, -0.0100])


In [19]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [23]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()  # Does the update

print(net.conv1.bias.grad)

tensor([-0.0049, -0.0055, -0.0059,  0.0172, -0.0007, -0.0123])
